# Notebook 02: Diffusion-Controlled Reactions

## 1. Introduction: Reactions in Solution

In the gas phase, molecules fly freely between collisions, and reaction rates are often determined by collision frequency and energy. In solution, the solvent changes everything:

| Gas Phase | Liquid Phase |
| :--- | :--- |
| Molecules fly freely | Molecules are crowded |
| Single collisions | **Cage Effect** (trapped encounter pairs) |
| Low density | High density, diffusion limits rate |
| $Z \approx 10^{34}$ m$^{-3}$s$^{-1}$ | $Z \approx 10^{36}$ m$^{-3}$s$^{-1}$ |

### The Cage Effect & Encounter Pairs
When reactants A and B meet in solution, they don't just collide once. They get trapped in a **solvent cage** and collide many times (an "encounter").

$$ A + B \underset{k_{-d}}{\overset{k_d}{\rightleftharpoons}} (AB) \xrightarrow{k_a} P $$

-   $k_d$: Rate of diffusing together to form the encounter pair $(AB)$.
-   $k_{-d}$: Rate of diffusing apart.
-   $k_a$: Rate of reaction within the cage.

![Solvent Cage](images/solvent_cage.png)

### Learning Objectives
1.  Understand Fick's Laws of diffusion.
2.  Calculate diffusion coefficients using the Stokes-Einstein equation.
3.  Derive the Smoluchowski limit for diffusion-controlled rates.
4.  Visualize the random walk of molecules in solution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
from scipy import constants
import ipywidgets as widgets

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 120,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'grid.alpha': 0.3
})

k_B = constants.Boltzmann
N_A = constants.Avogadro
pi = np.pi

print("Libraries loaded.")

### Visualizing the Cage Effect
The animation below simulates a "crowded" environment. The red particle is trapped by the blue solvent particles, forcing it to collide multiple times with neighbors before escaping.

In [ ]:
class CageEffectAnimator:
    """Simulate the cage effect in 2D"""
    
    def __init__(self, n_solvent=50):
        self.n_solvent = n_solvent
        self.fig, self.ax = plt.subplots(figsize=(6, 6))
        self.setup_simulation()
        
    def setup_simulation(self):
        self.ax.set_xlim(-5, 5)
        self.ax.set_ylim(-5, 5)
        self.ax.set_aspect('equal')
        self.ax.set_title('Solvent Cage Effect')
        self.ax.grid(False)
        
        # Solvent particles (blue)
        self.solvent_x = np.random.uniform(-5, 5, self.n_solvent)
        self.solvent_y = np.random.uniform(-5, 5, self.n_solvent)
        self.solvent_scat = self.ax.scatter(self.solvent_x, self.solvent_y, 
                                          c='lightblue', s=100, alpha=0.6)
        
        # Reactant particle (red)
        self.reactant_x = [0.0]
        self.reactant_y = [0.0]
        self.reactant_scat = self.ax.scatter([0], [0], c='red', s=150, edgecolors='black')
        
        # Trajectory line
        self.traj_line, = self.ax.plot([], [], 'r-', linewidth=1, alpha=0.5)
        
    def animate(self):
        def update(frame):
            # Random walk step for reactant
            step_size = 0.2
            dx = np.random.normal(0, step_size)
            dy = np.random.normal(0, step_size)
            
            # Simple hard-sphere repulsion from solvent
            new_x = self.reactant_x[-1] + dx
            new_y = self.reactant_y[-1] + dy
            
            # Check collisions with solvent (simplified)
            for i in range(self.n_solvent):
                dist = np.sqrt((new_x - self.solvent_x[i])**2 + (new_y - self.solvent_y[i])**2)
                if dist < 0.8: # Collision radius
                    # Bounce back
                    new_x = self.reactant_x[-1] - dx
                    new_y = self.reactant_y[-1] - dy
                    break
            
            # Boundary check
            if abs(new_x) > 5: new_x = self.reactant_x[-1]
            if abs(new_y) > 5: new_y = self.reactant_y[-1]
            
            self.reactant_x.append(new_x)
            self.reactant_y.append(new_y)
            
            # Keep trajectory short
            if len(self.reactant_x) > 50:
                self.reactant_x.pop(0)
                self.reactant_y.pop(0)
                
            self.reactant_scat.set_offsets(np.c_[new_x, new_y])
            self.traj_line.set_data(self.reactant_x, self.reactant_y)
            
            # Jiggle solvent slightly
            self.solvent_x += np.random.normal(0, 0.05, self.n_solvent)
            self.solvent_y += np.random.normal(0, 0.05, self.n_solvent)
            self.solvent_scat.set_offsets(np.c_[self.solvent_x, self.solvent_y])
            
            return self.reactant_scat, self.traj_line, self.solvent_scat
            
        anim = FuncAnimation(self.fig, update, frames=100, interval=50, blit=True)
        plt.close()
        return HTML(anim.to_jshtml())

print("\n🎬 Cage Effect Animation:")
animator = CageEffectAnimator(n_solvent=40)
display(animator.animate())

## 2. Diffusion and the Stokes-Einstein Equation

### 2.1 Fick's First Law
Diffusion is the net movement of particles from high concentration to low concentration. The flux $J$ (particles per area per second) is proportional to the concentration gradient:
$$ J = -D \frac{\partial c}{\partial x} $$
where $D$ is the **diffusion coefficient** (m$^2$/s).

### 2.2 Stokes-Einstein Equation
For a spherical particle of radius $R$ moving in a solvent of viscosity $\eta$, the drag force is $F = 6\pi\eta R v$. Einstein showed that the diffusion coefficient is related to thermal energy and this drag:
$$ D = \frac{k_B T}{6 \pi \eta R} $$

**Implications**:
-   Smaller molecules diffuse faster.
-   Higher temperature increases diffusion (more thermal energy).
-   Higher viscosity slows diffusion (more drag).

In [ ]:
def calculate_diffusion(T, eta_cP, R_nm):
    # Convert units
    eta = eta_cP * 1e-3  # Pa s (1 cP = 10^-3 Pa s)
    R = R_nm * 1e-9  # m
    
    # Stokes-Einstein
    D = (k_B * T) / (6 * pi * eta * R)
    
    print(f"Temperature: {T} K")
    print(f"Viscosity: {eta_cP} cP")
    print(f"Hydrodynamic Radius: {R_nm} nm")
    print(f"Diffusion Coefficient D: {D:.2e} m^2/s")
    print(f"D: {D * 1e9:.2f} x 10^-9 m^2/s (typical units)")

widgets.interact(calculate_diffusion, 
                 T=widgets.FloatSlider(min=200, max=400, step=10, value=298, description='T (K)'),
                 eta_cP=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.89, description='Viscosity (cP)'),
                 R_nm=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=0.5, description='Radius (nm)'));

## 3. The Smoluchowski Limit

What is the maximum possible rate for a reaction A + B $\rightarrow$ P?
Smoluchowski solved the diffusion equation for particles B diffusing towards a "sink" (particle A) that absorbs them immediately upon contact (at distance $R^* = R_A + R_B$).

### Derivation Sketch
1.  **Fick's First Law**: Flux $J = -D \frac{\partial [B]}{\partial r}$.
2.  **Steady-State Profile**: The concentration of B around A is given by:
    $$ [B](r) = [B]_{bulk} \left(1 - \frac{R^*}{r}\right) $$
    (Boundary condition: $[B] = 0$ at $r = R^*$).
3.  **Total Flux**: The total flux into the sphere of radius $R^*$ is:
    $$ \Phi = 4\pi (R^*)^2 J|_{R^*} = 4\pi R^* D [B]_{bulk} $$
4.  **Rate Constant**: The rate constant is the flux per unit concentration:
    $$ k_d = 4\pi R^* D $$

Converting to molar units:
$$ k_d = 4 \pi R^* (D_A + D_B) N_A $$

Using the Stokes-Einstein equation ($D = \frac{k_B T}{6\pi\eta R}$), we get a remarkable result:
$$ k_d \approx \frac{8RT}{3\eta} $$

**Key Insight**: The diffusion limit depends mainly on **solvent viscosity** and **temperature**, not on the size of the reactants!

In [ ]:
def smoluchowski_limit(D_sum_10_9, R_star_nm):
    D_sum = D_sum_10_9 * 1e-9  # m^2/s
    R_star = R_star_nm * 1e-9  # m
    
    # Rate constant in m^3 molecule^-1 s^-1
    kd_SI = 4 * pi * R_star * D_sum
    
    # Convert to M^-1 s^-1 (L mol^-1 s^-1)
    # Multiply by N_A to get per mole
    # Multiply by 1000 to convert m^3 to L
    kd_M = kd_SI * N_A * 1000
    
    print(f"Sum of Diffusion Coeffs: {D_sum:.2e} m^2/s")
    print(f"Reaction Distance R*: {R_star_nm} nm")
    print(f"Smoluchowski Limit kd: {kd_M:.2e} M^-1 s^-1")
    
    widgets.interact(smoluchowski_limit, 
                 D_sum_10_9=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=2.0, description='D_sum (10^-9)'),
                 R_star_nm=widgets.FloatSlider(min=0.1, max=2, step=0.1, value=0.5, description='R* (nm)'));

## 4. Activation vs. Diffusion Control

Real reactions involve both diffusion ($k_d$) and chemical activation ($k_a$). Applying the **Steady-State Approximation** to the encounter pair $(AB)$:

$$ \frac{d[(AB)]}{dt} = k_d[A][B] - k_{-d}[(AB)] - k_a[(AB)] \approx 0 $$

Solving for $[(AB)]$ and substituting into Rate $= k_a[(AB)]$, we get:
$$ k_{eff} = \frac{k_a k_d}{k_{-d} + k_a} $$

-   **Diffusion Control** ($k_a \gg k_{-d}$): $k_{eff} \approx k_d$. Rate depends on viscosity ($k \propto T/\eta$).
-   **Activation Control** ($k_a \ll k_{-d}$): $k_{eff} \approx K_{eq} k_a$. Rate depends on activation energy ($k \propto e^{-E_a/RT}$).

### Phase Diagram Explorer
The plot below shows the transition between the two regimes. The "Observed Rate" (black line) follows the slower of the two limits.

In [ ]:
def plot_rate_control(kd_log, ka_log):
    kd = 10**kd_log
    ka = 10**ka_log
    
    k_obs = (kd * ka) / (kd + ka)
    
    print(f"Diffusion Limit kd = {kd:.2e}")
    print(f"Activation Rate ka = {ka:.2e}")
    print(f"Observed Rate k_obs = {k_obs:.2e}")
    
    if kd < 0.1 * ka:
        print("Regime: Diffusion Controlled (limited by transport)")
    elif ka < 0.1 * kd:
        print("Regime: Activation Controlled (limited by chemistry)")
    else:
        print("Regime: Mixed Control")
        
    # Plot dependence on viscosity
    viscosity = np.logspace(-2, 1, 100) # relative viscosity
    # kd scales as 1/eta
    kd_vals = kd / viscosity
    
    k_obs_vals = (kd_vals * ka) / (kd_vals + ka)
    
    plt.figure(figsize=(8, 5))
    plt.loglog(viscosity, kd_vals, '--', label='Diffusion Limit (kd ~ 1/eta)')
    plt.loglog(viscosity, [ka]*len(viscosity), '--', label='Activation Limit (ka)')
    plt.loglog(viscosity, k_obs_vals, 'k-', linewidth=2, label='Observed Rate (k_obs)')
    plt.xlabel('Relative Viscosity')
    plt.ylabel('Rate Constant')
    plt.title('Effect of Viscosity on Reaction Rate')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(plot_rate_control, 
                 kd_log=widgets.FloatSlider(min=5, max=11, step=0.1, value=9, description='log(kd)'),
                 ka_log=widgets.FloatSlider(min=5, max=11, step=0.1, value=10, description='log(ka)'));

### The Material-Balance Equation
For a more general description of concentration changes in space and time (e.g., in a flow reactor or biological cell), we combine diffusion, convection, and reaction:

$$ \frac{\partial [J]}{\partial t} = \underbrace{D \frac{\partial^2 [J]}{\partial x^2}}_{Diffusion} - \underbrace{v \frac{\partial [J]}{\partial x}}_{Convection} - \underbrace{k_r [J]}_{Reaction} $$

In [ ]:
def solve_reaction_diffusion(D, kr, time_max):
    """Numerical solver for Reaction-Diffusion equation"""
    
    # Spatial domain
    L = 10.0
    nx = 100
    dx = L / nx
    x = np.linspace(-L/2, L/2, nx)
    
    # Time step (stability condition: dt < dx^2 / 2D)
    dt = 0.2 * dx**2 / D
    nt = int(time_max / dt)
    
    # Initial condition: Gaussian pulse
    u = np.exp(-x**2)
    
    # Store history for plotting
    history = [u.copy()]
    times = [0]
    
    # Time stepping loop
    for n in range(nt):
        un = u.copy()
        # Finite difference scheme
        # u_new = u + dt * (D * d2u/dx2 - kr * u)
        for i in range(1, nx-1):
            diffusion = D * (un[i+1] - 2*un[i] + un[i-1]) / dx**2
            reaction = -kr * un[i]
            u[i] = un[i] + dt * (diffusion + reaction)
            
        if n % (nt // 10) == 0:
            history.append(u.copy())
            times.append(n * dt)
            
    # Plotting
    plt.figure(figsize=(10, 6))
    
    # Plot evolution
    for i, u_state in enumerate(history):
        alpha = 0.2 + 0.8 * (i / len(history))
        plt.plot(x, u_state, label=f't={times[i]:.2f}', color=plt.cm.viridis(i/len(history)), linewidth=2)
        
    plt.xlabel('Position x')
    plt.ylabel('Concentration [J]')
    plt.title(f'Reaction-Diffusion Evolution (D={D}, kr={kr})')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

print("\n📊 Reaction-Diffusion Solver:")
widgets.interact(solve_reaction_diffusion, 
                 D=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Diffusion D'),
                 kr=widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.2, description='Reaction k'),
                 time_max=widgets.FloatSlider(min=1.0, max=10.0, step=1.0, value=5.0, description='Max Time'));

## Summary

1.  **Diffusion**: Molecules in solution move via random walks, described by Fick's laws and the Stokes-Einstein equation.
2.  **Smoluchowski Limit**: The maximum rate of reaction is determined by how fast reactants can diffuse together ($k_d \approx 10^{10}$ M$^{-1}$s$^{-1}$).
3.  **Rate Control**: Reactions can be diffusion-controlled (viscosity dependent) or activation-controlled (viscosity independent).
4.  **Material Balance**: Combines diffusion, convection, and reaction to describe concentration changes in space and time.